In [1]:
import numpy as np
import pandas as pd
import optuna
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error


In [2]:
df = pd.read_csv('../../train_belgrade.csv')

df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [3]:
def objective(trial):
    p = trial.suggest_int('p', 0, 3)
    d = trial.suggest_int('d', 0, 2)
    q = trial.suggest_int('q', 0, 3)
    P = trial.suggest_int('P', 0, 2)
    D = trial.suggest_int('D', 0, 1)
    Q = trial.suggest_int('Q', 0, 2)
    s = trial.suggest_categorical('s', [288, 2016])

    train_size = int(len(df) * 0.9)
    train, test = df.iloc[:train_size], df.iloc[train_size:]

    try:
        model = SARIMAX(train['Detections'], order=(p, d, q), seasonal_order=(P, D, Q, s), enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)

        forecast = model_fit.forecast(steps=len(test))

        mse = mean_squared_error(test['Detections'], forecast)
    except Exception as e:
        print(f"Error: {e}")
        mse = np.inf

    return mse


In [4]:

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)


[I 2024-02-20 08:34:34,354] A new study created in memory with name: no-name-ac795da8-bb84-48dd-b868-205a4ee2e354


[I 2024-02-20 08:44:40,757] Trial 0 finished with value: 221.18832922847793 and parameters: {'p': 1, 'd': 1, 'q': 0, 'P': 0, 'D': 0, 'Q': 1, 's': 288}. Best is trial 0 with value: 221.18832922847793.


In [ ]:

best_params = study.best_params
best_mse = study.best_value

print(f"Best Parameters: {best_params}")
print(f"Best MSE: {best_mse}")


Best Parameters: {'p': 1, 'd': 0, 'q': 1}
Best MSE: 45.6979698092569


In [ ]:

best_order = (best_params['p'], best_params['d'], best_params['q'])
best_seasonal_order = (best_params['P'], best_params['D'], best_params['Q'], best_params['s'])

model = SARIMAX(df['Detections'], order=best_order, seasonal_order=best_seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
model_fit = model.fit()

KeyError: 'P'